<a href="https://colab.research.google.com/github/faresbs/Representation-Learning/blob/master/Part3_VGGv3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import backend as K
from keras import optimizers as Kop
import os
import numpy as np
import shutil

In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
# dimensions of our images.
img_width, img_height = 64, 64

train_data_dir = '/content/drive/My Drive/Colab Notebooks/MachineLearning/newsplit/train'
validation_data_dir = '/content/drive/My Drive/Colab Notebooks/MachineLearning/newsplit/val'

nb_train_samples = 7999
nb_validation_samples = 2000
epochs = 150
batch_size = 32

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [0]:
print(os.listdir(train_data_dir))

['Dogs', 'Cats']


In [0]:
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=input_shape))
model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu'))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(filters=256, kernel_size=(3,3), activation='relu'))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(filters=256, kernel_size=(3,3), activation='relu'))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(filters=256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(filters=512, kernel_size=(3,3), activation='relu'))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(filters=512, kernel_size=(3,3), activation='relu'))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(filters=512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(filters=512, kernel_size=(3,3), activation='relu'))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(filters=512, kernel_size=(3,3), activation='relu'))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(filters=512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Flatten())
model.add(Dense(2048, activation='relu'))
model.add(Dense(2048, activation='relu'))
model.add(Dense(2, activation='softmax'))

SGD = Kop.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)

model.compile(loss='categorical_crossentropy',
              optimizer=SGD,
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_14 (ZeroPaddi (None, 66, 66, 3)         0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 64, 64, 64)        1792      
_________________________________________________________________
zero_padding2d_15 (ZeroPaddi (None, 66, 66, 64)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 64, 64, 64)        36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
zero_padding2d_16 (ZeroPaddi (None, 34, 34, 64)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 32, 32, 128)       73856     
__________

In [0]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.1,
    shear_range=0.1,
    horizontal_flip=True,
    rescale=1./255,
    fill_mode='nearest')
    
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    #classes = ['Dogs','Cats'],
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="categorical")

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    #classes = ['Dogs','Cats'],
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="categorical")

history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    verbose=1)

Found 15998 images belonging to 2 classes.
Found 4019 images belonging to 2 classes.
Epoch 1/150
 14/249 [>.............................] - ETA: 17:10 - loss: 0.6932 - acc: 0.5112

In [0]:
test_data_dir = '/content/drive/My Drive/Colab Notebooks/MachineLearning/testset'
print(os.listdir(test_data_dir))
print(len(os.listdir(test_data_dir)))

In [0]:
test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_width, img_height),
        class_mode='categorical',
        batch_size=1,  # only data, no labels
        shuffle=False)  # keep data in same order as labels

In [0]:
filenames = test_generator.filenames
print("filenames sample=",filenames[0:10])



In [0]:
nb_samples = len(filenames)
print("nb_samples=",nb_samples)

In [0]:
import re
def get_filename(s):
  return re.search('test/(.*).jpg', s).group(1)

In [0]:
predict = model.predict_generator(test_generator,steps = nb_samples, verbose=1)

In [0]:
print(predict)
final_predict = np.argmax(np.asarray(predict),axis=-1)

In [0]:
filenames2 = np.copy(np.array(filenames))

for i in range(len(filenames)):
  filenames2[i]=get_filename(filenames2[i])

filenames2 = np.asarray(filenames2, dtype = int)
print(filenames2[0:10])

In [0]:
labels = train_generator.class_indices
print(labels)

final_predict2 = np.asarray(final_predict, dtype=str)

for i in range(len(final_predict2)):
  if final_predict2[i]=='0':
    final_predict2[i] = 'Cat'
  else:
    final_predict2[i] = 'Dog'

  
predictions = np.dstack((filenames2,final_predict2))[0]

predictions2 = predictions[predictions[:,0].astype(int).argsort()]
print(predictions2)



In [0]:
np.savetxt("/content/drive/My Drive/Colab Notebooks/MachineLearning/submission.csv", np.asarray(predictions2), delimiter=",",fmt='%s', header='id,label')

In [0]:
import matplotlib.pyplot as plt
def visualize_training_history(history):
  # Training & validation accuracy values
  plt.plot(history.history['acc'])
  plt.plot(history.history['val_acc'])
  plt.title('Model accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Test'], loc='upper left')
  plt.show()

  # Training & validation loss values
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('Model loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Test'], loc='upper left')
  plt.show()

In [0]:
visualize_training_history(history)

In [0]:
from keras.models import load_model

model.save('VGG_150.h5')